# Compare sorters with some dataset generated with MEArec

Alessio Paolo Buccino author of [MEArec](https://github.com/alejoe91/MEArec) have generated some dataset

Here a comparison on the file **recordings_50cells_SqMEA-10-15um_10.0_10.0uV_2.h5**


In [1]:
import spiketoolkit as st
import spikeextractors as se

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

import time

%matplotlib notebook

21:55:33 [I] klustakwik KlustaKwik2 version 0.2.6


## Step 1 : open the dataset

The file have been downloaded in the current directory.


In [2]:
mearec_filename = 'recordings_50cells_SqMEA-10-15um_10.0_10.0uV_2.h5'
rec0  = se.MEArecRecordingExtractor(mearec_filename)
gt_sorting0 = se.MEArecSortingExtractor(mearec_filename)

AttributeError: 'Dataset' object has no attribute 'items'

## Step 2 : run all sorter

This run all sorter on all recording and store the results in a working folder.

In [3]:
# this cell is really verbose due to some sorter so switch off output console

recording_dict = {'rec0' : rec0}
sorter_list = ['tridesclous', 'herdingspikes','klusta', 'mountainsort4', 'spykingcircus']
# 'kilosort', 'kilosort2', 'ironclust' are not tested because I don't have matlab

working_folder = './working_folder'
t0 = time.perf_counter()
st.sorters.run_sorters(sorter_list, recording_dict, working_folder, engine=None)
t1 = time.perf_counter()
print('total run time', t1-t0)

NameError: name 'rec0' is not defined

## Step3 : collect Datatframe of comparison

This collect all results for all **(rec_name, sorter_name)** pair and apply the SortingComparison class for each.

This also concatenate performances in several dataframes.

In [4]:
ground_truths = {'toy_tetrode': gt_sorting0, 'toy_probe32': gt_sorting1}

comparisons, dataframes = st.comparison.gather_sorting_comparison(working_folder, ground_truths,use_multi_index=True)

NameError: name 'gt_sorting0' is not defined

## Step 4 : display tables

**dataframes** is a dict that contain several results formated in dataframes:
  * run_times : time for computing each one
  * perf_pooled_with_sum: performance with method **pooled_with_sum**
  * perf_pooled_with_average: performance with method **pooled_with_average**

In [5]:
dataframes['run_times']

NameError: name 'dataframes' is not defined

In [6]:
dataframes['perf_pooled_with_sum']

NameError: name 'dataframes' is not defined

In [7]:
dataframes['perf_pooled_with_average']

NameError: name 'dataframes' is not defined

## Step 5: inspect result by spiketrain

For finer exploration, we can also compute for all **(rec_name, sorter_name)** pair the performance with method=**by_spiketrain**.

This give a give result for each units of the ground truth.

In [8]:
for (rec_name, sorter_name), comp in comparisons.items():
    print(rec_name, sorter_name)
    perf = comp.get_performance(method='by_spiketrain')
    display(perf)

NameError: name 'comparisons' is not defined

## Step 6 : easy plot with seaborn

With the seaborn package, it is very easy to summary with plot dataframes.

Here some basic example for performances and run times.

In [9]:
run_times = dataframes['run_times'].reset_index()
fig, ax = plt.subplots()
sn.barplot(data=run_times, x='rec_name', y='run_time', hue='sorter_name', ax=ax)
ax.set_title('Run times')

NameError: name 'dataframes' is not defined

In [10]:
perfs = dataframes['perf_pooled_with_sum'].reset_index()
fig, ax = plt.subplots()
sn.barplot(data=perfs, x='rec_name', y='tp_rate', hue='sorter_name', ax=ax)
ax.set_title('True positive rate')
ax.set_ylim(0, 100)

NameError: name 'dataframes' is not defined

In [11]:
perfs = dataframes['perf_pooled_with_sum'].reset_index()
fig, ax = plt.subplots()
ax = sn.barplot(data=perfs, x='rec_name', y='accuracy', hue='sorter_name', ax=ax)
ax.set_title('accuracy')
ax.set_ylim(0, 100)

NameError: name 'dataframes' is not defined